# Automated ML on Heart Failure Dataset

Importing dependencies

In [1]:
import os
import joblib
import azureml.core
from azureml.core import Workspace, Experiment, Dataset, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from pprint import pprint # Used in printing automl model parameters
from azureml.core import Model # Used to get model information

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Initialize Workspace

Initialize a workspace object from persisted configuration. 

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-142887
aml-quickstarts-142887
southcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77


## Create an Azure ML experiment

Create an [Experiment](https://docs.microsoft.com/en-gb/azure/machine-learning/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace.

In [3]:
# Choose a name for the run history container in the workspace
experiment_name = 'heartfailure-automl'
experiment = Experiment(ws, experiment_name)

run = experiment.start_logging()

## Create or Attach an AmlCompute cluster

Create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for AutoML run

In [4]:
# Choose a name for your cluster
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
#cluster_name = "aml-cluster"
cluster_name = "project-automl"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f'{cluster_name} exists already')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    
    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
    
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

project-automl exists already
notebook142887 ComputeInstance Succeeded
project-automl AmlCompute Succeeded


## Dataset

### Overview

In this project, we are going to predict mortality due to heart failure with the use of AutoML. Heart failure is a common event caused by Cardiovascular diseases (CVDs), and it occurs when the heart cannot pump enough blood to meet the needs of the body.

The [Heart Failure Prediction](https://archive.ics.uci.edu/ml/datasets/Heart+failure+clinical+records) dataset is used as the training data for this task. It comprises of 299 heart failure patients and 12 features, which report clinical, body, and lifestyle information.

The task here is to train a binary classification model that predict the target column DEATH_EVENT, which indicates if the patient died or survived before the end of the follow-up period, based on the information provided by the other 11 columns (predictors). The time feature was dropped before training since we cannot get a time value for new patients after deployment. Prediction models based on these predictors, if accurate, can potentially be used to help hospitals in assessing the severity of patients with cardiovascular diseases.

In [6]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Heart-Failure Dataset"
description_text = "Heart-Failure Dataset for Captone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/PeacePeters/Heart-Failure-Prediction-using-AzureML/main/heart_failure.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [7]:
# Review the dataset result
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

The AutoML settings are:
1. The model is a classification task to predict mortality caused by heart failure.
2. The primary metric used is AUC weighted, which is more appropriate than accuracy since the dataset is moderately imbalanced (67.89% negative elements and 32.11% positive elements). 
3. A cross validation of 5 folds rather than 3 is used which gives a better performance. 
4. A 30 minutes timeout is specified to constrain usage. 
5. The maximum number of iterations to be executed in parallel during training is set to 5 max concurrent iterations. 

In [8]:
# Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task="classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             n_cross_validations=5,
                             debug_log="automl_errors.log",
                             **automl_settings
                            )

In [9]:
# Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heartfailure-automl,AutoML_c6857abb-a0b5-4c5f-ae11-eeba33cf46c3,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

In [10]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
heartfailure-automl,AutoML_c6857abb-a0b5-4c5f-ae11-eeba33cf46c3,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

{'runId': 'AutoML_c6857abb-a0b5-4c5f-ae11-eeba33cf46c3',
 'target': 'project-automl',
 'status': 'Completed',
 'startTimeUtc': '2021-04-17T00:43:24.693872Z',
 'endTimeUtc': '2021-04-17T01:25:32.7212Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'project-automl',
  'AMLSettingsJsonString': '{"path":null,"name":"heartfailure-automl","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-142887","workspace_name":"quick-starts-ws-142887","region":"southcentralus","compute_target":"project-automl","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"test_size":0.0,

## Best Model

In [11]:
# Get best run and model
best_run, best_model = remote_run.get_output()

In [12]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
heartfailure-automl,AutoML_c6857abb-a0b5-4c5f-ae11-eeba33cf46c3_105,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [13]:
# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

recall_score_weighted 0.8393785310734463
precision_score_macro 0.824556798401192
accuracy 0.8393785310734463
f1_score_weighted 0.8359945753349244
balanced_accuracy 0.8148643410852714
average_precision_score_micro 0.9241824826533982
AUC_weighted 0.9226163713547434
log_loss 0.42811259742649577
norm_macro_recall 0.6297286821705427
precision_score_micro 0.8393785310734463
recall_score_macro 0.8148643410852714
average_precision_score_weighted 0.9352476103800835
matthews_correlation 0.6370694049823383
f1_score_micro 0.8393785310734463
precision_score_weighted 0.8568664469679866
average_precision_score_macro 0.9151985863556545
weighted_accuracy 0.8560851677689973
f1_score_macro 0.8063664974189777
AUC_micro 0.9214179035398512
AUC_macro 0.9226163713547434
recall_score_micro 0.8393785310734463
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_c6857abb-a0b5-4c5f-ae11-eeba33cf46c3_105/confusion_matrix
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_c6857abb-a0b5-4c5f-ae11-eeba3

In [14]:
print('Best Run Id: ' + best_run.id,
     'Best Model Name: ' + best_run.properties['model_name'])
print('\n AUC_weighted:', best_run_metrics['AUC_weighted'])

Best Run Id: AutoML_c6857abb-a0b5-4c5f-ae11-eeba33cf46c3_105 Best Model Name: AutoMLc6857abba105

 AUC_weighted: 0.9226163713547434


In [15]:
best_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               scale_pos_weight=1,
                                                                                               seed=None,
                                                                                               silent=None,
                                                               

In [16]:
best_model._final_estimator

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('69',
                                           Pipeline(memory=None,
                                                    steps=[('minmaxscaler',
                                                            MinMaxScaler(copy=True,
                                                                         feature_range=(0,
                                                                                        1))),
                                                           ('randomforestclassifier',
                                                            RandomForestClassifier(bootstrap=True,
                                                                                   ccp_alpha=0.0,
                                                                                   class_weight='balanced',
                                                                                   criterion

In [17]:
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['69', '61', '79', '94', '66', '60', '98', '64', '49', '19'],
 'weights': [0.07142857142857142,
             0.14285714285714285,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.14285714285714285,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.21428571428571427]}

69 - minmaxscaler
{'copy': True, 'feature_range': (0, 1)}

69 - randomforestclassifier
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 0.2,
 'm

In [ ]:
best_run.get_environment()

In [18]:
best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":4,"CurrentNodeCount":4}',
 'mlflow.source.type': 'JOB',
 'mlflow.source.name': 'automl_driver.py',
 'ensembled_iterations': '[69, 61, 79, 94, 66, 60, 98, 64, 49, 19]',
 'ensembled_algorithms': "['RandomForest', 'RandomForest', 'GradientBoosting', 'RandomForest', 'GradientBoosting', 'GradientBoosting', 'RandomForest', 'GradientBoosting', 'LightGBM', 'XGBoostClassifier']",
 'ensemble_weights': '[0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.21428571428571427]',
 'best_individual_pipeline_score': '0.9175457733480987',
 'best_individual_iteration': '69',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

## Model Deployment

In [23]:
# Register the model
myModel = best_run.register_model(model_path='outputs/model.pkl', model_name=experiment_name+"-model",
                   tags={'Training context':'AutoML', 'type': 'Classification'})
myModel

Model(workspace=Workspace.create(name='quick-starts-ws-142887', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-142887'), name=heartfailure-automl-model, id=heartfailure-automl-model:2, version=2, tags={'Training context': 'AutoML', 'type': 'Classification'}, properties={})

In [20]:
# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

heartfailure-automl-model version: 1
	 Training context : AutoML
	 type : Classification


hyperdrive-heart-failure-best-model version: 1
	 Training context : Parameterized SKLearn Estimator
	 type : Classification
	 AUC_weighted : 0.8166666666666667




In [21]:
best_run.download_file('outputs/model.pkl', './model.pkl')

# Download scoring file
best_run.download_file('outputs/scoring_file_v_1_0_0.py', './score.py')

best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

In [24]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

env = Environment.get(ws, "AzureML-AutoML")
inference_config = InferenceConfig(entry_script='./score.py', environment=env)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                               memory_gb=1,
                                               enable_app_insights=True, 
                                               tags={'name': 'aci-cluster', 'framework': 'AutoML'},
                                               description='Heart Failure Predictor Web Service')

service = Model.deploy(workspace=ws, 
                           name=experiment_name+"-service",
                           models=[myModel], 
                           inference_config=inference_config, 
                           deployment_config=aci_config,
                           overwrite=True)

service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-17 01:43:26+00:00 Creating Container Registry if not exists.
2021-04-17 01:43:27+00:00 Registering the environment.
2021-04-17 01:43:27+00:00 Use the existing image.
2021-04-17 01:43:28+00:00 Generating deployment configuration.
2021-04-17 01:43:28+00:00 Submitting deployment to compute.
2021-04-17 01:43:31+00:00 Checking the status of deployment heartfailure-automl-service..
2021-04-17 01:49:30+00:00 Checking the status of inference endpoint heartfailure-automl-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [25]:
print(service.scoring_uri)

print(service.swagger_uri)

http://b0f36165-afd0-403d-a7f4-56024382d28b.southcentralus.azurecontainer.io/score
http://b0f36165-afd0-403d-a7f4-56024382d28b.southcentralus.azurecontainer.io/swagger.json


In [58]:
import json
import requests

# import test data
test_df = df.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('DEATH_EVENT')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})
print(test_sample)

{"data": [{"age": 67.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 50, "high_blood_pressure": 0, "platelets": 263358.03, "serum_creatinine": 1.18, "serum_sodium": 137, "sex": 1, "smoking": 1, "time": 76}, {"age": 52.0, "anaemia": 0, "creatinine_phosphokinase": 132, "diabetes": 0, "ejection_fraction": 30, "high_blood_pressure": 0, "platelets": 218000.0, "serum_creatinine": 0.7, "serum_sodium": 136, "sex": 1, "smoking": 1, "time": 112}, {"age": 65.0, "anaemia": 0, "creatinine_phosphokinase": 167, "diabetes": 0, "ejection_fraction": 30, "high_blood_pressure": 0, "platelets": 259000.0, "serum_creatinine": 0.8, "serum_sodium": 138, "sex": 0, "smoking": 0, "time": 186}, {"age": 70.0, "anaemia": 1, "creatinine_phosphokinase": 59, "diabetes": 0, "ejection_fraction": 60, "high_blood_pressure": 0, "platelets": 255000.0, "serum_creatinine": 1.1, "serum_sodium": 136, "sex": 0, "smoking": 0, "time": 85}, {"age": 50.0, "anaemia": 1, "creatinine_phosphokinase":

In [59]:
# predict using the deployed model
result = service.run(test_sample)
print(result)

{"result": [0, 0, 0, 0, 1]}


In [60]:
# Set the content type
headers = {'Content-type': 'application/json'}

response = requests.post(service.scoring_uri, test_sample, headers=headers)

# Print results from the inference
print(response.text)

"{\"result\": [0, 0, 0, 0, 1]}"


In [61]:
# Print original labels
print(label_df)

In [ ]:
print(service.get_logs())

In [ ]:
service.delete()